In [ ]:
import tensorflow as tf

from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import h5py
import numpy as np
from PIL import Image

# Load CERN abstract images
f = h5py.File("/content/drive/MyDrive/CERN ONN/000.h5",'r')

p = h5py.File("/content/drive/MyDrive/CERN ONN/001.h5",'r')

# print(type(hdf))
for key in f.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key]))

for key in p.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(p[key]))

import numpy as np
import matplotlib.pyplot as plt

a_group_key = list(f.keys())[0]
data1 = list(f[a_group_key])
data1 = np.array(data1)
print(data1.shape)
plt.imshow(data1[6,:,:,1])



b_group_key = list(f.keys())[1]
labels1 = list(f[b_group_key])
labels1 = np.array(labels1)
print(labels1.shape)

import numpy as np
import matplotlib.pyplot as plt

a_group_key2 = list(p.keys())[0]
data2 = list(p[a_group_key2])
data2 = np.array(data2)
print(data2.shape)
plt.imshow(data2[8,:,:,1])



b_group_key2 = list(p.keys())[1]
labels2 = list(p[b_group_key2])
labels2 = np.array(labels2)
print(labels2.shape)


u = h5py.File("/content/drive/MyDrive/CERN ONN/002.h5",'r')

a_group_key3 = list(u.keys())[0]
data3 = list(u[a_group_key3])
data3 = np.array(data3)
print(data3.shape)
plt.imshow(data3[8,:,:,1])



b_group_key3 = list(u.keys())[1]
labels3 = list(u[b_group_key3])
labels3 = np.array(labels3)
print(labels3.shape)

e = h5py.File("/content/drive/MyDrive/CERN ONN/003.h5",'r')

a_group_key4 = list(e.keys())[0]
data4 = list(e[a_group_key4])
data4 = np.array(data4)
print(data4.shape)
plt.imshow(data4[8,:,:,1])



b_group_key4 = list(e.keys())[1]
labels4 = list(e[b_group_key4])
labels4 = np.array(labels4)
print(labels4.shape)

##
x = h5py.File("/content/drive/MyDrive/CERN ONN/004.h5",'r')

a_group_key5 = list(x.keys())[0]
data5 = list(x[a_group_key5])
data5 = np.array(data5)
print(data5.shape)
plt.imshow(data5[8,:,:,1])



b_group_key5 = list(x.keys())[1]
labels5 = list(x[b_group_key5])
labels5 = np.array(labels5)
print(labels5.shape)

##

m = h5py.File("/content/drive/MyDrive/CERN ONN/005.h5",'r')

a_group_key6 = list(m.keys())[0]
data6 = list(m[a_group_key6])
data6 = np.array(data6)
print(data6.shape)
plt.imshow(data6[8,:,:,1])



b_group_key6 = list(m.keys())[1]
labels6 = list(m[b_group_key6])
labels6 = np.array(labels6)
print(labels6.shape)



Images
<class 'h5py._hl.dataset.Dataset'>
Labels
<class 'h5py._hl.dataset.Dataset'>
Images
<class 'h5py._hl.dataset.Dataset'>
Labels
<class 'h5py._hl.dataset.Dataset'>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = np.concatenate((data1, data2, data3, data4, data5, data6), axis = 0)
labels = np.concatenate((labels1, labels2, labels3, labels4, labels5, labels6), axis = 0)

(4500, 150, 94, 5)
(4500, 3)


In [ ]:
#Select images that will be used to train and test model
just_tri = np.zeros((4500, 150, 94,1))
tri_labels = np.zeros(4500)
count = 0
count1 = 0
count2 = 0
# DO I NEED COUNT?
for i in range(0,4500):
    if labels[i,0] == 1:
        tri_labels[count] = 0
        just_tri[count,:,:,0] = data[i,:,:,1]
        count += 1
        count1 += 1
    elif labels[i,1] == 1:
        tri_labels[count] = 1
        just_tri[count,:,:,0] = data[i,:,:,1]
        count += 1
        count2 +=1

In [ ]:
just_tri = just_tri[:3000]

tri_labels = tri_labels[:3000]

In [ ]:
def reshape_spectrogram_to_square(spectrogram, target_side_length):

     rows, cols = spectrogram.shape

     pad_rows = max(0, target_side_length - rows)
     pad_cols = max(0, target_side_length - cols)


     padded_spectrogram = np.pad(spectrogram, ((round(pad_rows/2), pad_rows - round(pad_rows/2)), (round(pad_cols/2), pad_cols - round(pad_cols/2))), mode='constant', constant_values=0)
     cropped_spectrogram = np.flip(padded_spectrogram[:target_side_length, :target_side_length], axis = 0)

     return cropped_spectrogram

In [ ]:
# Pad Abstract images with zeros
just_tri_reshaped = np.zeros((3000,150,150))
for i in range(0,3000):
    img = just_tri[i,:,:,0]
    img = reshape_spectrogram_to_square(img,150)
    just_tri_reshaped[i,:,:] = img

In [ ]:

just_tri = just_tri[0:3000]
tri_labels = tri_labels[0:3000]
print(just_tri.shape)

(3000, 150, 94, 1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(just_tri_reshaped, tri_labels, test_size=0.1, random_state=42)

print(X_train.shape)


(2700, 150, 150)


In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 150, 150, 1)  # Add the channel dimension (1 for grayscale)
X_test = X_test.reshape(-1, 150, 150, 1)


In [ ]:
from tensorflow import keras
# Set up model
model = keras.models.Sequential([
    keras.layers.Conv2D(1, (10,10), activation='relu', input_shape=(150,150,1)),
    keras.layers.MaxPooling2D(3,3),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(learning_rate=0.0001),
             metrics=['acc'])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore weights from the epoch with the best value of the monitored quantity
)

In [ ]:
#begin training
batch_size = 5
num_classes = 1
epochs = 60
history = model.fit(X_train, y_train.reshape(2700,1),
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/30
2700/2700 [==============================] - 72s 26ms/step - loss: 0.6931 - acc: 0.5122 - val_loss: 0.6918 - val_acc: 0.4933
Epoch 2/30
2700/2700 [==============================] - 70s 26ms/step - loss: 0.6860 - acc: 0.5374 - val_loss: 0.6778 - val_acc: 0.5200
Epoch 3/30
2700/2700 [==============================] - 68s 25ms/step - loss: 0.6630 - acc: 0.5763 - val_loss: 0.6489 - val_acc: 0.5367
Epoch 4/30
2700/2700 [==============================] - 66s 24ms/step - loss: 0.6266 - acc: 0.6463 - val_loss: 0.6059 - val_acc: 0.6900
Epoch 5/30
2700/2700 [==============================] - 67s 25ms/step - loss: 0.5839 - acc: 0.7219 - val_loss: 0.5627 - val_acc: 0.7867
Epoch 6/30
2700/2700 [==============================] - 68s 25ms/step - loss: 0.5438 - acc: 0.7578 - val_loss: 0.5264 - val_acc: 0.7733
Epoch 7/30
2700/2700 [==============================] - 66s 24ms/step - loss: 0.5098 - acc: 0.7693 - val_loss: 0.5003 - val_acc: 0.7800
Epoch 8/30
2700/2700 [==========================

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



10/10 [==============================] - 3s 282ms/step - loss: 0.4014 - acc: 0.8200
Test Loss: 0.4014
Test Accuracy: 82.00%


['loss', 'acc']

In [ ]:
weights = model.get_weights()

In [ ]:
def relu(X):
   return np.maximum(0,X)
def sigmoid(X):
   return 1/(1+np.exp(-X))

In [ ]:
#Optical convolution simulation
def fft_forward(img,w, b, hparameters1):

    l = img.shape[1]
    pa = (l - 10) // 2
    f2 = np.pad(w.reshape(10,10), (pa,pa))
    img = img.reshape(l,l)

    if l % 2 == 0:
        l = l-1
        img = img[:l,:l]
        f2 = f2[:l,:l]

        f = np.fft.fft2(img)

        f21 = np.fft.fft2(f2)


        aaa = f * f21
        final = np.fft.ifft2(aaa)

        zeros = np.zeros((l,l))
        zeros[1:l // 2+1,1:l // 2+1] = final.real[l // 2+1:,l // 2+1:]
        zeros[1:l // 2+1,l // 2+1:] = final.real[l // 2+1:,1:l // 2+1]
        zeros[l // 2+1:,1:l // 2+1] = final.real[1:l // 2+1,l // 2+1:]
        zeros[l // 2+1:,l // 2+1:] = final.real[1:l // 2+1,1:l // 2+1]

        zeros = zeros[1:l,1:l] + b

    else:
        f = np.fft.fft2(img)
        f21 = np.fft.fft2(f2)
        aaa = f * f21
        final = np.fft.ifft2(aaa)

        zeros = np.zeros((l,l))
        zeros[1:l // 2+1,1:l // 2+1] = final.real[l // 2+1:,l // 2+1:]
        zeros[1:l // 2+1,l // 2+1:] = final.real[l // 2+1:,1:l // 2+1]
        zeros[l // 2+1:,1:l // 2+1] = final.real[1:l // 2+1,l // 2+1:]
        zeros[l // 2+1:,l // 2+1:] = final.real[1:l // 2+1,1:l // 2+1]

        zeros = zeros[1:l-1,1:l-1] + b
    return zeros

In [ ]:
#Function to create version of model with just convolutional layer and version without convolutonal layer
def extract_layers(main_model, starting_layer_ix, ending_layer_ix):

  new_model = Sequential()
  for ix in range(starting_layer_ix, ending_layer_ix + 1):
    curr_layer = main_model.get_layer(index=ix)

    new_model.add(curr_layer)
  return new_model

new_model = extract_layers(model, 1, 4)

In [ ]:
def new_fft_model(img, weights, hparameters1, hparameters2):
    fft = relu(fft_forward(img, weights[0], weights[1], hparameters1))
    print(np.array(fft).shape)
    fft = fft[4:145, 4:145]
    print(fft.shape)
    tup = new_model.predict(fft.reshape(1,141,141,1))

    return tup

In [ ]:
hparameters1 = {
    'stride':1,
    'pad':0
}
hparameters2 = {
    'stride':2,
    'f':2
}
weights = model.get_weights()
print(type(weights))

<class 'list'>


# Evaluating with extracted model

In [ ]:
from PIL import Image

# Assuming X_test contains images
fft_out = []
count = 0
for i in range(len(X_test)):
    # Resize image using PIL
    img_pil = Image.fromarray(X_test[i,:,:,0])
    img_pil = img_pil.resize((150, 150))

    # Convert resized image to numpy array and reshape
    img = np.array(img_pil).reshape(1, 150, 150, 1)

    output1 = new_fft_model(img, weights, hparameters1, hparameters2)


    if output1[0] >= 0.5:
        pred = 1
        fft_out.append(1)
    else:
        pred = 0
        fft_out.append(0)

    if pred == y_test[i]:
        count += 1

print(count/len(X_test))

(148, 148)
(141, 141)
1/1 [==============================] - 0s 232ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 31ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 35ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 32ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 32ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 45ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 29ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 27ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 23ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 22ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 22ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 21ms/step
(148, 148)
(141, 141)
1/1 [==============================] - 0s 21ms/step
(148, 148)
(141, 141)
1/1 [==========